![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>Tutorial  <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

This notebook provides an introductory tutorial for [**MuJoCo** physics](https://github.com/google-deepmind/mujoco#readme), using the native Python bindings.

<!-- Copyright 2021 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->

# All imports

In [72]:
%load_ext autoreload
%autoreload 2

# Set up GPU rendering.
import distutils.util
import os
import subprocess
# # if subprocess.run('nvidia-smi').returncode:
# #   raise RuntimeError(
# #       'Cannot communicate with GPU. '
# #       'Make sure you are using a GPU Colab runtime. '
# #       'Go to the Runtime menu and select Choose runtime type.')

# # Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# # This is usually installed as part of an Nvidia driver package, but the Colab
# # kernel doesn't install its driver via APT, and as a result the ICD is missing.
# # (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
# NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
# if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
#   with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
#     f.write("""{
#     "file_format_version" : "1.0.0",
#     "ICD" : {
#         "library_path" : "libEGL_nvidia.so.0"
#     }
# }
# """)

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
  import mujoco.viewer # Also have to import this to trigger the installation of the viewer.
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np
# from scipy.spatial.transform import Rotation as Robj
from scipy.optimize import curve_fit, fsolve, least_squares
from scipy.signal import medfilt
from scipy.stats import linregress
from utils.helper_fns import *
from utils.render_opts import *
import utils.robot_controller as robot_controller
import utils.model_selection as model_selection
# import utils.com_estimation as com
from utils.com_estimation import F_model, theta_model, align_zeros

# Graphics and plotting.
import mediapy as media
import matplotlib.pyplot as plt
import contextlib

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)
# Set matplotlib font size
fonts = {'size' : 20}
plt.rc('font', **fonts)
%matplotlib inline
# %matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl
Checking that the installation succeeded:
Installation successful.


## Full Toppling Simulation

In [107]:

# ======================== Toggle visualization here =========================
VIZ = True   # set to False to record video without showing the viewer
# ============================================================================

## Let's recall the model to reset the simulation
model, data, com_gt = model_selection.select_model('../assets/table_push.xml', 'box_exp')
## Setup based on robot model
irb = robot_controller.controller(model, data)

## Set robot just in front of payload (same orientation as home position (facing +x))
T_init = np.eye(4)
T_init[0:3, 3] = np.array([0.95, 0.0, 0.3])
q_init = irb.IK(T_init, method=2, damping=0.5, max_iters=1000) # DLS method
irb.set_pose(q=q_init)

## The end pose we want to reach FOR POSITION CONTROL (format: 4x4 matrix)
T_end = T_init.copy()
T_end[0, 3] += 0.1  # Move EE forward by 15 cm in x direction
target_q = irb.IK(T_end, method=2, damping=0.5, max_iters=1000)  # DLS method

# Initialize log
log = []
episode_done = False
workspace_bounds = {
    "xmin": -1.5, "xmax": 1.5,
    "ymin": -0.5, "ymax": 0.5,
    "zmin": 0.2, "zmax": 1.0
}
workspace_bounds = None

# Simulation parameters
traj_duration = 6.0 # seconds
run_duration = traj_duration + 2.0  # seconds

rv = RendererViewerOpts(model, data, vis=VIZ)

# =========================== Visualization Loop =================================
# with rv.viewer_ctx as viewer:
with rv: # enters viewer if vis=True, sets viewer opts, and readies offscreen renderer for video capture
    while rv.viewer_is_running() and not episode_done and not irb.stop:
        # irb.check_topple()                          # Check for payload topple condition

        # Use new step control primitive
        result = irb.move_ee_delta(
            dx_world=[0.01, 0, 0],
            n_steps=5,
            max_force=30.0,
            force_window=3,
            check_ellipsoid=True,
            )

        # Log snapshot AFTER motion primitive
        snapshot = irb.capture_state(
            q_desired=result["q_target"] if result["q_target"] is not None else irb.data.qpos(irb.joint_idx),
            )
        log.append(snapshot)
        # mujoco.mj_step(model, data)                 # Step the simulation

        # Check high-level termination
        episode_done, term_reason = irb.check_termination(
            workspace_bounds=workspace_bounds,
            tilt_thresh_deg=50.0,
            max_time=run_duration,
        )

        rv.sync()                               # Update the interactive viewer (if present)
        rv.capture_frame_if_due(data)

# No need to close renderer since context manager handles it

# tip_edge_pairs = np.asarray(tip_edge_pairs, dtype=float)            # Convert to numpy array for easier indexing

print('\n\n======================================')
print(f'Simulation ended in {data.time:.2f} s due to {term_reason}.')
print('======================================')

# Play the movie
media.show_video(rv.frames, fps=rv.framerate)

This shouldnt be printing
Loading environment '../assets/table_push.xml' with object 'box_exp'...
Model loaded successfully.

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************
Doing nothing for now

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************

IK finished, robot state restored.
**********************************



This browser does not support the video tag.


In [49]:
print(f'Number of logged snapshots: {len(log)}')
print(log[0])

Number of logged snapshots: 6541
{'time': 0.001, 'ee_pos': array([0.951, 0.   , 0.3  ]), 'ee_quat': array([-0., -0.,  0.,  1.]), 'q': array([ 0.   ,  0.553,  0.732,  0.   , -1.286, -0.   ]), 'payload_pos': array([1. , 0. , 0.2]), 'payload_quat': array([0., 0., 0., 1.]), 'payload_rpy': array([0., 0., 0.]), 'ft_wrench': array([-0.   , -0.   , -0.333,  0.   ,  0.001,  0.333]), 'q_desired': array([ 0.   ,  0.553,  0.732,  0.   , -1.286, -0.   ]), 'obj_mass': 0.635, 'obj_com_world': array([1. , 0. , 0.2]), 'obj_friction': None}
